# <center>Segmenting and Clustering Neighborhoods in Toronto</center>
<i><center> By: Phil Murphy </center><i>

### **Create Notebook and Import Libraries**


New Library - BeautifulSoup - http://beautiful-soup-4.readthedocs.io/en/latest/

In [1]:
import pandas as pd 
import numpy as np 
from bs4 import BeautifulSoup 
import requests 

print('Libraries Imported')

Libraries Imported


### **Scrape Data from Website**

From: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [2]:
from urllib.request import urlopen
import csv

print('CSV Imported')

CSV Imported


In [3]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')

print('Data Scraped by BeautifulSoup')

Data Scraped by BeautifulSoup


In [4]:
table = soup.find('table',{'class':'wikitable sortable'})
#table

table_rows = table.find_all('tr')
#table_rows

In [5]:
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighborhood'])
df = df[~df['PostalCode'].isnull()]  #to filter out bad rows

print(df.head(10))

   PostalCode           Borough                                 Neighborhood
1         M1A      Not assigned                                 Not assigned
2         M2A      Not assigned                                 Not assigned
3         M3A        North York                                    Parkwoods
4         M4A        North York                             Victoria Village
5         M5A  Downtown Toronto                    Regent Park, Harbourfront
6         M6A        North York             Lawrence Manor, Lawrence Heights
7         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government
8         M8A      Not assigned                                 Not assigned
9         M9A         Etobicoke      Islington Avenue, Humber Valley Village
10        M1B       Scarborough                               Malvern, Rouge


### **Create Pandas Dataframe from Scraped Data**

In [6]:
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180 entries, 1 to 180
Data columns (total 3 columns):
PostalCode      180 non-null object
Borough         180 non-null object
Neighborhood    180 non-null object
dtypes: object(3)
memory usage: 5.6+ KB


(180, 3)

### **Clean Dataframe**
Ignore all records with borough value "not assigned"


In [7]:
import pandas
import requests
from bs4 import BeautifulSoup
website_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_text,'lxml')

table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pandas.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighborhood'])
df = df[~df['PostalCode'].isnull()]  #to filter Not Assigned rows

df.head(15)

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M8A,Not assigned,Not assigned
9,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
10,M1B,Scarborough,"Malvern, Rouge"


In [8]:
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
df1 = df.reset_index()
df1.head()

,index,PostalCode,Borough,Neighborhood
0,3,M3A,North York,Parkwoods
1,4,M4A,North York,Victoria Village
2,5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,6,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [10]:
df1.info()
df1.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 4 columns):
index           103 non-null int64
PostalCode      103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
dtypes: int64(1), object(3)
memory usage: 3.3+ KB


(103, 4)

### **More than one neighborhood can exist in the one postal code.**

We will combine neighborhoods seperated by a comma into one row

In [11]:
df2= df1.groupby('PostalCode').agg(lambda x: ','.join(x))

df2.head()

,Borough,Neighborhood
PostalCode,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [12]:
df2.info()
df2.shape

<class 'pandas.core.frame.DataFrame'>
Index: 103 entries, M1B to M9W
Data columns (total 2 columns):
Borough         103 non-null object
Neighborhood    103 non-null object
dtypes: object(2)
memory usage: 2.4+ KB


(103, 2)

### **If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.**

In [13]:
df2.loc[df2['Neighborhood']=="Not assigned",'Neighborhood']=df2.loc[df2['Neighborhood']=="Not assigned",'Borough']
df2.tail()

,Borough,Neighborhood
PostalCode,,
M9N,York,Weston
M9P,Etobicoke,Westmount
M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
M9W,Etobicoke,"Northwest, West Humber - Clairville"


In [14]:
df3 = df2.reset_index()
df3.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [15]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
PostalCode      103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
dtypes: object(3)
memory usage: 2.5+ KB


In [16]:
df3.shape

(103, 3)